In [1]:
# Importation des librairies
from webscraping.ws_fonctions import *


In [4]:
notes_presse_vs_spect = get_presse_vs_spect(2029, 2)
notes_presse_vs_spect.head(6)

,Auteurs,Note
0,Spectateurs,"2,8"
